In [ ]:
import glob
import os

from astropy.io import fits
from astroquery.mast import Observations
from wfc3tools import calwf3

In [ ]:
propid='12527'
obserid='IBRU01040'
#propid='12959'
#obserid='IC1707020'
#propid='13297'
#obserid='ICAU22020'
#propid='13804'
#obserid='ICMX05030'
#propid='14164'
#obserid='ICW806050'
#propid='14610'
#obserid='IDB616030'
#propid='15275'
#obserid='IDN011030'
#propid='15083'
#obserid='IDLE02010'
#propid='15275'
#obserid='IDN073030'
#propid='15857'
#obserid='IE0O61LYQ'

obsTable = Observations.query_criteria(proposal_id=[propid], obs_id=[obserid])
product_list = Observations.get_product_list(obsTable)
BM = (product_list['productSubGroupDescription']  == 'RAW') 
product_list = product_list[BM]

product_list.show_in_notebook(display_length=20)

In [ ]:
#2015 - icmx05qlq
myID = product_list['obsID'][2:3]
print(myID)
download = Observations.download_products(myID,mrp_only=False,productSubGroupDescription=['RAW'])


In [ ]:
local_path = download['Local Path'][0]
#filename = '2016_test_01_18_2022_' + local_path[29:]
filename = '2012_yearly_low_10_18_2022_' + local_path[29:]
os.system('cp {} {}'.format(local_path, filename))

In [ ]:
raw_files = glob.glob('*raw.fits') 
# Check that the postflash file was updated: 
#for file in glob.glob('*raw.fits'):
for file in glob.glob(filename):
    hdu = fits.open(file)
    print(hdu[0].header['SHUTRPOS'])
    hdu.close()


#for file in raw_files:
 #   command_line_input = 'crds bestrefs --files {:} --sync-references=1 --update-bestrefs'.format(file)
 #   os.system(command_line_input)

In [ ]:
filename

In [ ]:
from astropy.io import fits 
#calfile='2021_B_final_cal.fits'
calfile='2012_pf_B_low_curr.fits'
with fits.open(calfile,mode='update') as hdu:
    h = hdu[0].header
   # h.set('FILETYPE', 'FLSHFILE')
    #FILETYPE= 'POST FLASH'         / type of data found in data file   
    h.set('FILETYPE', 'POST FLASH')

In [ ]:

# Check the current postflash values for the raw files you want to test with:
#for file in glob.glob('*raw.fits'):
for file in glob.glob(filename):
    hdu = fits.open(file)
    print(hdu[0].header['FLSHFILE'])
    hdu.close()

In [ ]:
# Open each raw and update header to have the new postflash file

#for file in glob.glob('*raw.fits'):
for file in glob.glob(filename):
    with fits.open(file,mode='update') as hdu:
        h = hdu[0].header
        h.set('FLSHFILE', calfile)    
        h.set('FLSHCORR', 'PERFORM')

In [ ]:
# Check that the postflash file was updated: 
#for file in glob.glob('*raw.fits'):
for file in glob.glob(filename):
    hdu = fits.open(file)
    print(hdu[0].header['FLSHFILE'])
    hdu.close()

In [ ]:
# Now run the darks through calibration with the new postflash file: 
for rawfits in glob.glob(filename):
    calwf3(rawfits)